In [53]:
import pandas as pd
import gzip
import struct
import itertools


PATH_RATINGS = '../data/mcauley_amazon/video_games/ratings_Video_Games.csv'
PATH_META = '../data/mcauley_amazon/video_games/meta_Video_Games.json.gz'
PATH_IMG_EMBED = '../data/mcauley_amazon/video_games/image_features_Video_Games.b'

In [102]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

def readImageFeatures(path):
    f = open(path, 'rb')
    while True:
        asin = f.read(10)
        if asin == b'': break
        ret = [str(asin,'utf-8')]
        for i in range(4096):
            ret.append(struct.unpack('f', f.read(4))[0])
        yield ret

In [8]:
%%time
meta_df = getDF(PATH_META).set_index('asin')

CPU times: user 6.06 s, sys: 88 ms, total: 6.15 s
Wall time: 6.16 s


In [30]:
meta_df.to_msgpack('../data/snap_amazon/video_games/meta.msg', compress='zlib')

In [24]:
%%time
ratings_df = pd.read_csv(PATH_RATINGS,
                         header=None, names=['user_id', 'asin', 'rating', 'timestamp'])
ratings_df['timestamp'] = pd.to_datetime(ratings_df['timestamp'], unit='s')

CPU times: user 696 ms, sys: 68 ms, total: 764 ms
Wall time: 766 ms


In [31]:
ratings_df.to_msgpack('../data/snap_amazon/video_games/ratings.msg', compress='zlib')

In [103]:
em_gen = readImageFeatures(PATH_IMG_EMBED)

In [104]:
%%time
img_emb_df = pd.DataFrame(list(em_gen))

CPU times: user 2min 6s, sys: 6.78 s, total: 2min 13s
Wall time: 2min 13s


In [107]:
img_emb_df.set_index(0, inplace=True)
img_emb_df.index.name = 'asin'

In [117]:
img_emb_df.astype('float32').to_msgpack('../data/snap_amazon/video_games/img_embs.msg', compress='zlib')

In [118]:
ratings_df

,user_id,asin,rating,timestamp
0,AB9S9279OZ3QO,0078764343,5.0,2013-07-07
1,A24SSUT5CSW8BH,0078764343,5.0,2013-08-24
2,AK3V0HEBJMQ7J,0078764343,4.0,2013-07-04
3,A10BECPH7W8HM7,043933702X,5.0,2014-07-10
4,A2PRV9OULX1TWP,043933702X,5.0,2013-12-04
5,AE7GUHCDQQ4UI,043933702X,1.0,2013-04-17
6,A48ABFDDRMKI8,043933702X,5.0,2013-07-19
7,A26B0P6K95SIKW,0439339960,3.0,2010-11-01
8,AZ3UWOC8QSO6C,0439339987,5.0,2013-04-25
9,A182S3ANC0W7DL,0439342260,4.0,2012-12-19


In [39]:
meta_df.loc[ratings_df.asin.value_counts().nlargest(10).index]

,related,price,description,imUrl,salesRank,categories,title,brand
B00DJFIMW6,"{'also_bought': ['B009UX2YAC', 'B00IKZX1ZI', '...",NaN,NaN,NaN,NaN,"[[Apps for Android, Games], [Video Games, Fire...",NaN,NaN
B00BGA9WK2,"{'bought_together': ['B00BGA9X9W', 'B00CXCCI8A...",28.12,,http://ecx.images-amazon.com/images/I/41omR-LT...,{'Video Games': 13},"[[Electronics, Television & Video, Blu-ray Pla...",NaN,NaN
B00FAX6XQC,"{'also_bought': ['B007Z4AKQI', 'B00AIUUXHC', '...",NaN,NaN,NaN,NaN,"[[Apps for Android, Games], [Video Games, Fire...",NaN,NaN
B009KS4XRO,"{'also_bought': ['B009WJNXAE', 'B00E5GJ7QS', '...",NaN,NaN,NaN,NaN,"[[Apps for Android, Games], [Video Games, Fire...",NaN,NaN
B002VBWIP6,"{'also_bought': ['B00F4CEHNK', 'B00F4CEWOY', '...",59.90,"With an Xbox LIVE Gold membership, take your X...",http://ecx.images-amazon.com/images/I/415TMngo...,{'Video Games': 17},"[[Gift Cards, Gift Cards], [Video Games, Xbox ...",NaN,NaN
B0055SWM08,"{'also_bought': ['B006OOHYIC', 'B00CT2P6VA', '...",NaN,NaN,NaN,NaN,"[[Apps for Android, Games], [Video Games, Fire...",NaN,NaN
B00CSR2J9I,"{'also_bought': ['B009UX2YAC', 'B00AIUUXHC', '...",NaN,NaN,NaN,NaN,"[[Apps for Android, Games], [Video Games, Fire...",NaN,NaN
B0015AARJI,"{'bought_together': ['B00IA2NYT4', 'B000Y1MOIQ...",54.99,The Dualshock 3 wireless controller for the Pl...,http://ecx.images-amazon.com/images/I/41E1q8uk...,{'Video Games': 99},"[[Electronics, Accessories & Supplies, Audio &...",NaN,NaN
B00178630A,"{'bought_together': ['B00FENJVOE', 'B000ZKA0J6...",39.99,,http://ecx.images-amazon.com/images/I/41kXCp%2...,{'Video Games': 1053},"[[Video Games, PC, Games]]",NaN,NaN
B000FKBCX4,"{'bought_together': ['B001NPCTDC', 'B003X0LG1Q...",32.89,"Create universal wonder in Spore, an exciting ...",http://ecx.images-amazon.com/images/I/517S6KnR...,{'Video Games': 368},"[[Video Games, Mac, Games], [Video Games, PC, ...",NaN,NaN
